## Mask R-CNN Model

In [1]:
import cv2
import numpy as np
import colorsys
import time
from time import time as timer

cv2.__version__

'3.4.0'

In [2]:
import os
import sys
#import random
#import math
#import numpy as np
#import skimage.io
import matplotlib
import matplotlib.pyplot as plt
    
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
    
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
import mrcnn.utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

/home/cfchen/Github36/Mask_RCNN-master


/home/cfchen/anaconda3/envs/MaskRCNN/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

## Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")
    
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()
    
print(ROOT_DIR)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9

## Configurations

## Load Model

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

print(ROOT_DIR)

/home/cfchen/Github36/Mask_RCNN-master


## Functions

In [6]:
#############
# Functions #
#############

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    np.random.shuffle(colors)
    return colors

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image



### Masked

In [7]:
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    #print(n_instances)
    print('Frame Number {}: Found {} boxes for {}'.format(count - 1, n_instances, 'img'))

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        '''
        Detect Only Person
        '''
        '''        
        if ids[i] != 1:
            continue
        '''
        '''
        Detect Only Person
        '''        
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        #image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255,0), 5)
        #image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)
        
        '''
        #print(n_instances)
        TLBR = '{} {} {} {}'.format(x1, y1, x2, y2)
        captionLaIdSc = '{} {} {:.2f}'.format(label, ids[i], score) if score else label        
        print("{}  {}  {}".format(count - 1 , captionLaIdSc, TLBR))
        #print(ids)
        #print(ids[i])
        '''
        #print(n_instances)
        
        #with open('outputEis.txt', 'a') as f:
        with open('outputEis.csv', 'a') as f:
            TLBR = '{}, {}, {}, {}'.format(x1, y1, x2, y2)
            captionLaIdSc = '{}, {}, {:.2f},'.format(label, ids[i], score) if score else label
            print("{},  {}  {}".format(count - 1 , captionLaIdSc, TLBR), file=f)
        #f.close()
        
    return image

capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('temp.avi',fourcc, 30.0, (960, 540))     # resize image half

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

start = timer()

count = 0
while(True):
    ret, frame = capture.read() 
    
    if ret == True:

        count += 1
        
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half

        results = model.detect([frame], verbose=0)

        r = results[0]
    
        frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

        '''
        print('Frame NO.    {}: Found {} boxes for {}'.format(count - 1, r['rois'].shape[0], 'img'))
        #print('{}  {}  {}'.format(print(r['scores']), r['class_ids'], r['rois']))
        #print(r['scores'])           # Confidence
        #print(r['rois'])             # Bounding Boxes 
        #print(r['class_ids'])        # Class IDs        
        '''
        
        out.write(frame)
        
        cv2.imshow('frame', frame)
        
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break

    else:
        break
        
print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

Video Dim: 1920.0  X  1080.0
Frame Number 0: Found 8 boxes for img
Frame Number 1: Found 9 boxes for img
Frame Number 2: Found 11 boxes for img
Frame Number 3: Found 11 boxes for img
Frame Number 4: Found 10 boxes for img
Frame Number 5: Found 7 boxes for img
Frame Number 6: Found 10 boxes for img
Frame Number 7: Found 11 boxes for img
Frame Number 8: Found 8 boxes for img
Frame Number 9: Found 10 boxes for img
Frame Number 10: Found 8 boxes for img
Frame Number 11: Found 8 boxes for img
Frame Number 12: Found 8 boxes for img
Frame Number 13: Found 7 boxes for img
Frame Number 14: Found 9 boxes for img
Frame Number 15: Found 9 boxes for img
Frame Number 16: Found 9 boxes for img
Frame Number 17: Found 8 boxes for img
Frame Number 18: Found 8 boxes for img
Frame Number 19: Found 9 boxes for img
Frame Number 20: Found 8 boxes for img
Frame Number 21: Found 9 boxes for img
Frame Number 22: Found 10 boxes for img
Frame Number 23: Found 9 boxes for img
Frame Number 24: Found 12 boxes for im

Frame Number 204: Found 8 boxes for img
Frame Number 205: Found 9 boxes for img
Frame Number 206: Found 9 boxes for img
Frame Number 207: Found 7 boxes for img
Frame Number 208: Found 8 boxes for img
Frame Number 209: Found 8 boxes for img
Frame Number 210: Found 8 boxes for img
Frame Number 211: Found 9 boxes for img
Frame Number 212: Found 9 boxes for img
Frame Number 213: Found 9 boxes for img
Frame Number 214: Found 10 boxes for img
Frame Number 215: Found 9 boxes for img
Frame Number 216: Found 6 boxes for img
Frame Number 217: Found 6 boxes for img
Frame Number 218: Found 6 boxes for img
Frame Number 219: Found 6 boxes for img
Frame Number 220: Found 7 boxes for img
Frame Number 221: Found 7 boxes for img
Frame Number 222: Found 9 boxes for img
Frame Number 223: Found 7 boxes for img
Frame Number 224: Found 7 boxes for img
Frame Number 225: Found 6 boxes for img
Frame Number 226: Found 6 boxes for img
Frame Number 227: Found 5 boxes for img
Frame Number 228: Found 8 boxes for img

### Compute mAP @ IoU=50 on Batch of Images

### Precision-Recall

In [ ]:
'''
#print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))
#dataset.image_info[1:3]

print(dataset.num_classes)
print(dataset.num_images)
print(len(dataset.image_info[2]))
print(dataset.image_info[2]['id'])
print(dataset.image_info[2]['source'])
print(dataset.image_info[2]['path'])
print(dataset.image_info[2]['width'])
print(dataset.image_info[2]['height'])
#print(dataset.image_info[2]['annotations'])
#print(dataset.image_info[2])
print('')

print(dataset.class_names[8])
print(dataset.class_ids)
print('')

print(dataset.class_info[80])
print(dataset.class_info[80]['source'])
dataset.class_info[80]['name']
'''